# 02_GNN_training_evaluation
Build PyG data object from processed_long_rainfall_v2.csv and demo .npy

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
os.makedirs('demo', exist_ok=True)

proc = pd.read_csv('processed_long_rainfall_v2.csv')
print('Processed shape:', proc.shape)
proc.head()

In [ ]:
# build small demo npy: node features = rainfall, edge_index = connect nodes on same date within neighbor distance
meta = proc.head(500).copy().reset_index(drop=True)
meta['node_id'] = range(len(meta))
x = meta[['rainfall']].to_numpy(dtype=float)
# build edges
edges = []
for d, group in meta.groupby('date'):
    pts = group[['node_id','lat','lon']].values
    for i, (nid, lat, lon) in enumerate(pts):
        for j, (nid2, lat2, lon2) in enumerate(pts):
            if nid!=nid2:
                d2 = (lat-lat2)**2 + (lon-lon2)**2
                if d2 < 0.0005:
                    edges.append([int(nid), int(nid2)])
edge_index = np.array(edges).T if len(edges)>0 else np.zeros((2,0), dtype=int)
# target y: inflow if available, else 0
y = meta['inflow'].fillna(0).to_numpy(dtype=float)
demo = {'x':x, 'edge_index':edge_index, 'y':y, 'meta_table': meta.to_dict(orient='records')}
np.save('demo/sample_dataset_v2.npy', demo)
print('Saved demo/sample_dataset_v2.npy')

In [ ]:
print('Use src/train.py to train a GNN; see src/ for model and training loop.')